In [1]:
import sys
sys.path.append("..")

import pandas as pd
from reader import generic_reader

!pip install streamlit

In [2]:
df_movies = generic_reader.read_csv_file_to_data_frame("../movie_data/movies.csv")


In [3]:
empty_directors = df_movies[df_movies["director_person_ids"].apply(lambda x: len(x) == 0)]
empty_directors

,adult,backdrop_path,budget,homepage,movie_id,imdb_id,origin_country,original_language,original_title,overview,...,vote_average,vote_count,director_person_ids,genre_ids,production_company_ids,collection_id,cast_person_ids,cast_credit_ids,crew_person_ids,crew_credit_ids


In [4]:
empty_directors = df_movies[df_movies["cast_person_ids"].apply(lambda x: len(x) == 0)]
empty_directors

,adult,backdrop_path,budget,homepage,movie_id,imdb_id,origin_country,original_language,original_title,overview,...,vote_average,vote_count,director_person_ids,genre_ids,production_company_ids,collection_id,cast_person_ids,cast_credit_ids,crew_person_ids,crew_credit_ids


In [5]:
len(df_movies["director_person_ids"].dropna())

2801

In [6]:
len(df_movies["cast_person_ids"])

2801

In [7]:
len(df_movies["cast_person_ids"].dropna())

2801